<a href="https://colab.research.google.com/github/acram002/AI-Driven-Recipe-Suggestion-System/blob/main/deepseekNew2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TRAINING BLOCK
# ✅ STEP 1: Install dependencies
!pip install -q unsloth datasets bitsandbytes accelerate peft

# ✅ STEP 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ STEP 3: Load model with Unsloth
from unsloth import FastLanguageModel
import torch

model_name = "deepseek-ai/deepseek-llm-7b-base"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=1024,
    dtype=torch.bfloat16,
    load_in_4bit=True,
    device_map="auto"
)

# ✅ STEP 4: Apply LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none"
)

# ✅ STEP 5: Load and clean dataset
import pandas as pd, ast
from datasets import Dataset

df = pd.read_csv('/content/drive/MyDrive/full_dataset.csv')
df = df[['NER', 'ingredients', 'directions']].dropna()
df = df.sample(n=30000, random_state=42)

# Clean functions
def clean_ingredient_list(ingredient_str):
    try:
        ingredients = ast.literal_eval(ingredient_str) if isinstance(ingredient_str, str) else ingredient_str
        cleaned = []
        for item in ingredients:
            if any(unit in item.lower() for unit in ['tortilla', 'egg', 'clove', 'slice']):
                parts = item.split()
                if len(parts) > 1 and parts[0][0].isdigit():
                    cleaned.append(" ".join(parts[1:]))
                else:
                    cleaned.append(item)
            elif any(unit in item.lower() for unit in ['lb', 'pound', 'lbs', 'kg']):
                cleaned.append(item.replace("6 lbs", "1.5 lbs").replace("5 lbs", "1.5 lbs"))
            else:
                cleaned.append(item)
        return cleaned
    except Exception:
        return []

def clean_response(text):
    cutoff_phrases = [
        "Note:", "Nutritional", "Serves", "Source:", "Enjoy!", "This is a vegan dish",
        "From Food Network", "submitted by", "adapted from", "let me know", "Thanks for visiting"
    ]
    for phrase in cutoff_phrases:
        idx = text.lower().find(phrase.lower())
        if idx != -1:
            return text[:idx].strip()
    return text.strip()

# Apply cleaning
df['NER'] = df['NER'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith("[") else x)
df['ingredients'] = df['ingredients'].apply(clean_ingredient_list)
df['response'] = df.apply(
    lambda row: f"Ingredients:\n{chr(10).join(row['ingredients'])}\n\nInstructions:\n" + "\n".join(ast.literal_eval(row['directions']))
    if isinstance(row['directions'], str) and row['directions'].startswith("[") else str(row['directions']),
    axis=1
)
df['response'] = df['response'].apply(clean_response)
df = df[df['response'].str.len() > 100]
df = df[df['NER'].apply(lambda x: isinstance(x, list) and len(x) >= 3)]

# Create prompt-response and convert to dataset
df['prompt'] = df['NER'].apply(
    lambda x: f"<|user|> Generate a recipe using ONLY the following ingredients: {', '.join(x)}. Return only the title, ingredients, and step-by-step instructions. <|assistant|>"
)
df['text'] = df['prompt'] + "\n" + df['response']

dataset = Dataset.from_pandas(df[['text']])
dataset = dataset.train_test_split(test_size=0.05)
train_data = dataset["train"]
eval_data = dataset["test"]

# ✅ STEP 6: Tokenization
max_length = 1024

def tokenize(example):
    model_inputs = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=max_length
    )
    model_inputs["labels"] = [
        [(token if token != tokenizer.pad_token_id else -100) for token in input_ids]
        for input_ids in model_inputs["input_ids"]
    ]
    return model_inputs

tokenized_train = train_data.map(tokenize, batched=True, remove_columns=["text"])
tokenized_eval = eval_data.map(tokenize, batched=True, remove_columns=["text"])

# ✅ STEP 7: Setup Trainer
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/deepseek-7b-recipe-lora2",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    logging_steps=10,
    save_strategy="steps",
    save_steps=250,
    save_total_limit=2,
    bf16=True,
    optim="paged_adamw_8bit",
    warmup_steps=5,
    weight_decay=0.01,
    lr_scheduler_type="linear",
    learning_rate=2e-4,
    report_to="none",
    remove_unused_columns=False
)

model.enable_input_require_grads()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer
)

# ✅ STEP 8: Train
trainer.train()

# ✅ STEP 9: Save
model.save_pretrained("/content/drive/MyDrive/deepseek-7b-recipe-lora2")
tokenizer.save_pretrained("/content/drive/MyDrive/deepseek-7b-recipe-lora2")

print("\n✅ Training complete. Model saved to: /content/drive/MyDrive/deepseek-7b-recipe-lora2")


In [ ]:
# TEST GENERATION: USED GPU
# ✅ STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ STEP 2: Install Unsloth
!pip install -q unsloth

# ✅ STEP 3: Load base model + fine-tuned LoRA adapter
from unsloth import FastLanguageModel
import torch, re

base_model = "deepseek-ai/deepseek-llm-7b-base"
adapter_path = "/content/drive/MyDrive/deepseek-7b-recipe-lora2"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_model,
    max_seq_length=1024,
    dtype=torch.bfloat16,
    load_in_4bit=True,
    device_map="auto"
)

model.load_adapter(adapter_path)

# ✅ STEP 4: Test prompts
test_prompts = [
    "chicken, rice, broccoli, soy sauce, garlic",
    "ice cream, banana, chocolate syrup, whipped cream",
    "eggs, cheese, spinach, tomato, tortilla",
    "ground beef, tomato sauce, spaghetti",
    "beef, potatoes, carrots, onion, broth, bay leaf",
    "tofu, mushrooms, soy sauce, garlic, sesame oil"
]

stop_phrases = ["Note:", "Enjoy!", "submitted by", "Serves", "Source", "let me know", "Thanks for visiting", ".com", "@", "Photo credit"]

# ✅ STEP 5: Evaluation
def evaluate_recipe(prompt, recipe):
    score = {"completeness": 0, "creativity": 0}
    ingredients_list = [i.strip().lower() for i in prompt.split("ingredients:")[-1].split(".")[0].split(",")]

    has_ingredients = "ingredients" in recipe.lower()
    has_instructions = "instructions" in recipe.lower()
    used_ingredients = [i for i in ingredients_list if i in recipe.lower()]
    coverage = len(used_ingredients) / len(ingredients_list) if ingredients_list else 0

    score["completeness"] = round((int(has_ingredients) + int(has_instructions) + (1 if coverage > 0.8 else 0)) / 3 * 10, 1)

    verbs = re.findall(r'\b(bake|roast|saute|simmer|boil|fry|grill|steam|whisk|marinate|blend|microwave|stir)\b', recipe.lower())
    adjectives = re.findall(r'\b(crispy|creamy|savory|sweet|spicy|tender|rich|zesty|golden|charred)\b', recipe.lower())

    if len(set(verbs)) >= 3: score["creativity"] += 4
    if len(set(adjectives)) >= 2: score["creativity"] += 3
    if len(recipe.split()) > 100: score["creativity"] += 3

    return score

# ✅ STEP 6: Run inference
for i, ingredients in enumerate(test_prompts, 1):
    prompt = f"<|user|> Generate a structured recipe using ONLY these ingredients: {ingredients}. Provide just the title, ingredients list, and step-by-step instructions. <|assistant|>"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=300,
        do_sample=True,
        top_k=40,
        top_p=0.92,
        temperature=0.7,
        repetition_penalty=1.15,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    recipe = decoded.split("<|assistant|>")[-1].strip()

    for phrase in stop_phrases:
        if phrase in recipe:
            recipe = recipe.split(phrase)[0].strip()

    scores = evaluate_recipe(prompt, recipe)
    print(f"\n🧪 Test Case {i}: {ingredients}")
    print("=== Generated Recipe ===\n", recipe)
    print("📊 Completeness Score:", scores["completeness"], "/ 10")
    print("🎨 Creativity Score:", scores["creativity"], "/ 10")
    print("=" * 60)


In [ ]:
# TEST: DID NOT RUN ON CPU, GO TO NEXT BLOCK
# ✅ STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ STEP 2: Install Unsloth
!pip install -q unsloth

# ✅ STEP 3: Load base model + fine-tuned LoRA adapter
from unsloth import FastLanguageModel
import torch, re

base_model = "deepseek-ai/deepseek-llm-7b-base"
adapter_path = "/content/drive/MyDrive/deepseek-7b-recipe-lora2"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_model,
    max_seq_length=1024,
    dtype=torch.bfloat16,
    load_in_4bit=True,
    device_map="auto"
)

model.load_adapter(adapter_path)

# ✅ STEP 4: Get user input
ingredients = input("📝 Enter ingredients (comma-separated): ").strip()

stop_phrases = ["Note:", "Enjoy!", "submitted by", "Serves", "Source", "let me know", "Thanks for visiting", ".com", "@", "Photo credit"]

# ✅ STEP 5: Evaluation
def evaluate_recipe(prompt, recipe):
    score = {"completeness": 0, "creativity": 0}
    ingredients_list = [i.strip().lower() for i in prompt.split("ingredients:")[-1].split(".")[0].split(",")]

    has_ingredients = "ingredients" in recipe.lower()
    has_instructions = "instructions" in recipe.lower()
    used_ingredients = [i for i in ingredients_list if i in recipe.lower()]
    coverage = len(used_ingredients) / len(ingredients_list) if ingredients_list else 0

    score["completeness"] = round((int(has_ingredients) + int(has_instructions) + (1 if coverage > 0.8 else 0)) / 3 * 10, 1)

    verbs = re.findall(r'\b(bake|roast|saute|simmer|boil|fry|grill|steam|whisk|marinate|blend|microwave|stir)\b', recipe.lower())
    adjectives = re.findall(r'\b(crispy|creamy|savory|sweet|spicy|tender|rich|zesty|golden|charred)\b', recipe.lower())

    if len(set(verbs)) >= 3: score["creativity"] += 4
    if len(set(adjectives)) >= 2: score["creativity"] += 3
    if len(recipe.split()) > 100: score["creativity"] += 3

    return score

# ✅ STEP 6: Run inference
prompt = f"<|user|> Generate a structured recipe using ONLY these ingredients: {ingredients}. Provide just the title, ingredients list, and step-by-step instructions. <|assistant|>"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

output = model.generate(
    **inputs,
    max_new_tokens=300,
    do_sample=True,
    top_k=40,
    top_p=0.92,
    temperature=0.7,
    repetition_penalty=1.15,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id
)

decoded = tokenizer.decode(output[0], skip_special_tokens=True)
recipe = decoded.split("<|assistant|>")[-1].strip()

for phrase in stop_phrases:
    if phrase in recipe:
        recipe = recipe.split(phrase)[0].strip()

scores = evaluate_recipe(prompt, recipe)
print("\n=== Generated Recipe ===\n", recipe)
print("📊 Completeness Score:", scores["completeness"], "/ 10")
print("🎨 Creativity Score:", scores["creativity"], "/ 10")
print("=" * 60)


In [ ]:
# TEST: USED CPU SUCCESSFULLY
# ✅ STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ STEP 2: Install normal libraries
!pip install -q transformers peft

# ✅ STEP 3: Load model correctly
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Paths
base_model_name = "deepseek-ai/deepseek-llm-7b-base"
adapter_path = "/content/drive/MyDrive/deepseek-7b-recipe-lora2"

# ✅ Load base model (CPU, float32)
print("🚀 Loading base model...")
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float32,
    device_map="cpu",
)

# ✅ Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# ✅ Apply LoRA adapter
print("🔗 Applying LoRA...")
model = PeftModel.from_pretrained(model, adapter_path)
model = model.to("cpu")

print("✅ Model ready!")

# ✅ STEP 4: Get user input
ingredients = input("📝 Enter ingredients (comma-separated): ").strip()

stop_phrases = ["Note:", "Enjoy!", "submitted by", "Serves", "Source", "let me know", "Thanks for visiting", ".com", "@", "Photo credit"]

# ✅ STEP 5: Evaluation function
import re

def evaluate_recipe(prompt, recipe):
    score = {"completeness": 0, "creativity": 0}
    ingredients_list = [i.strip().lower() for i in prompt.split("ingredients:")[-1].split(".")[0].split(",")]

    has_ingredients = "ingredients" in recipe.lower()
    has_instructions = "instructions" in recipe.lower()
    used_ingredients = [i for i in ingredients_list if i in recipe.lower()]
    coverage = len(used_ingredients) / len(ingredients_list) if ingredients_list else 0

    score["completeness"] = round((int(has_ingredients) + int(has_instructions) + (1 if coverage > 0.8 else 0)) / 3 * 10, 1)

    verbs = re.findall(r'\b(bake|roast|saute|simmer|boil|fry|grill|steam|whisk|marinate|blend|microwave|stir)\b', recipe.lower())
    adjectives = re.findall(r'\b(crispy|creamy|savory|sweet|spicy|tender|rich|zesty|golden|charred)\b', recipe.lower())

    if len(set(verbs)) >= 3: score["creativity"] += 4
    if len(set(adjectives)) >= 2: score["creativity"] += 3
    if len(recipe.split()) > 100: score["creativity"] += 3

    return score

# ✅ STEP 6: Run inference
prompt = f"<|user|> Generate a structured recipe using ONLY these ingredients: {ingredients} The user prefe {user_pref} Allergies are {allergies}, do not. The user previously liked these recipes {liked_recipes} . Provide just the title, ingredients list, and step-by-step instructions. <|assistant|>"

inputs = tokenizer(prompt, return_tensors="pt").to("cpu")

output = model.generate(
    **inputs,
    max_new_tokens=300,
    do_sample=True,
    top_k=40,
    top_p=0.92,
    temperature=0.7,
    repetition_penalty=1.15,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)

decoded = tokenizer.decode(output[0], skip_special_tokens=True)
recipe = decoded.split("<|assistant|>")[-1].strip()

for phrase in stop_phrases:
    if phrase in recipe:
        recipe = recipe.split(phrase)[0].strip()

scores = evaluate_recipe(prompt, recipe)

print("\n=== Generated Recipe ===\n", recipe)
print("📊 Completeness Score:", scores["completeness"], "/ 10")
print("🎨 Creativity Score:", scores["creativity"], "/ 10")
print("=" * 60)


In [ ]:
# ATTEMPTED TO MERGE LORA AND BASE MODEL: DIDNT WORK
# ✅ STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ STEP 2: Install Unsloth
!pip install -q unsloth

# ✅ STEP 3: Import
from unsloth import FastLanguageModel
import torch

# ✅ STEP 4: Load base model + adapter
base_model_path = "deepseek-ai/deepseek-llm-7b-base"
lora_adapter_path = "/content/drive/MyDrive/deepseek-7b-recipe-lora2"
save_merged_path = "/content/drive/MyDrive/deepseek-7b-merged-recipe-model"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_model_path,
    max_seq_length=1024,
    dtype=torch.bfloat16,
    load_in_4bit=True,
    device_map="auto"
)

# ✅ STEP 5: Load LoRA adapter (already patched)
model.load_adapter(lora_adapter_path)

# ✅ STEP 6: Merge adapter into base model
model = model.merge_and_unload()

# ✅ STEP 7: Save the final merged model
model.save_pretrained(save_merged_path)
tokenizer.save_pretrained(save_merged_path)

print("\n✅ Merge complete! Merged model saved to:", save_merged_path)
